In [ ]:
import lancedb

In [ ]:
uri = 'data/new1'

In [ ]:
db = lancedb.connect(uri)

In [ ]:
data =[ 
    
    {'vector' : [1,1], 'item' : 'food', 'price':150},
    {'vector' : [2,2], 'item' : 'oil', 'price':20},
    {'vector' : [3,3], 'item' : 'milk', 'price':100}
    
]

In [ ]:
data[1]['vector']

In [ ]:
tbl = db.create_table('Table1', data = data)

In [ ]:
import pyarrow as pa

In [ ]:
##schema = pa.schema([pa.field('vector', pa.list_(pa.float32(), list_size =2 ))])
schema = pa.schema([
    pa.field('vector', pa.list_(pa.float32(), list_size=2)),
    pa.field('item', pa.string()),
    pa.field('price', pa.float32())  # or pa.int32() if price is always integer
])


In [ ]:
tbl2 = db.create_table('table2' , schema=schema)
##db.drop_table("table2")

In [ ]:
tbl2 = db.open_table('table2')

In [ ]:
print(db.list_tables())

In [ ]:
tbl2.add(data)

In [ ]:
tbl2

In [ ]:
tbl.search([5,5]).limit(1).to_pandas()

In [ ]:
tbl.search('milk').limit(1).to_pandas()

In [ ]:
pip install tantivy

In [ ]:
tbl.search('milk').limit(1).to_pandas()

In [ ]:
tbl.create_fts_index("item")

In [ ]:
tbl

In [ ]:
tbl.search('milk').limit(2).to_pandas()

In [ ]:
tbl.search('item == milk').limit(2).to_pandas()

In [ ]:
tbl.delete('item == "milk"')

In [ ]:
tbl.search([5,5]).limit(2).to_pandas()

In [ ]:
db.drop_table('table2')

In [ ]:
data2 = [
    
    {'vector': [1,5], 'item': 'rice', 'price' : 1000}
    
]

In [ ]:
tbl.add(data2)

In [ ]:
tbl.search([1,5]).limit(3).to_pandas()

In [ ]:
tbl.delete( 'price = 1000')

In [ ]:
tbl.search([1,5]).limit(3).to_pandas()

In [ ]:
tbl.update(where="price = 150", values={"vector": [10, 10]})

In [ ]:
tbl.search([1,5]).limit(3).to_pandas()